In [55]:
import plotly.express as px

data_series = {'Total Nonfarm': 'CES0000000001',
               'Manufacturing' : 'CES3000000001',
               'Total Private': 'CES0500000001',
               'Government': 'CES9000000001',
               'Construction': 'CES2000000001',
               'Oil and Gas Extraction': 'CES1021100001',
               'Professional and Business Services':'CES6000000001',
               'Leisure and Hospitality':'CES7000000001',
               'Other Services': 'CES8000000001',
               'Mining and Logging': 'CES1000000001',
               'Financial Activities': 'CES5500000001',
               'Private Education and Health Services': 'CES6500000001',
               'Information' : 'CES5000000001',
               'Retail Trade': 'CES4200000001',
              }

series = {value:key for key, value in data_series.items()}
series_id = [key for key in series]

household_employment = 'LNS12000000'
'''
 Fabricated Metal Product Manufacturing Average Weekly Hours of All Employees, Seasonally Adjusted - CES3133200002
 Paper Manufacturing Average Weekly Hours of All Employees, Seasonally Adjusted - CES3232200002
 Private Education and Health Services Average Weekly Hours of All Employees, Not Seasonally Adjusted - CEU6500000002
 Printing and Related Support Activities Average Weekly Hours of All Employees, Seasonally Adjusted - CES3232300002
 Total Private Employment, Not Seasonally Adjusted - CEU0500000001
 Total Private Average Hourly Earnings of Production and Nonsupervisory Employees, Seasonally Adjusted - CES0500000008

 Total Private Average Hourly Earnings of All Employees, Seasonally Adjusted - CES0500000003
 Total Nonfarm Employment, Not Seasonally Adjusted - CEU0000000001
 Total Private Average Weekly Hours of All Employees, Seasonally Adjusted - CES0500000002
 Primary Metal Manufacturing Average Weekly Hours of All Employees, Seasonally Adjusted - CES3133100002
 Textile Mills Average Weekly Hours of All Employees, Seasonally Adjusted - CES3231300002

 '''

'\n Fabricated Metal Product Manufacturing Average Weekly Hours of All Employees, Seasonally Adjusted - CES3133200002\n Paper Manufacturing Average Weekly Hours of All Employees, Seasonally Adjusted - CES3232200002\n Private Education and Health Services Average Weekly Hours of All Employees, Not Seasonally Adjusted - CEU6500000002\n Printing and Related Support Activities Average Weekly Hours of All Employees, Seasonally Adjusted - CES3232300002\n Total Private Employment, Not Seasonally Adjusted - CEU0500000001\n Total Private Average Hourly Earnings of Production and Nonsupervisory Employees, Seasonally Adjusted - CES0500000008\n\n Total Private Average Hourly Earnings of All Employees, Seasonally Adjusted - CES0500000003\n Total Nonfarm Employment, Not Seasonally Adjusted - CEU0000000001\n Total Private Average Weekly Hours of All Employees, Seasonally Adjusted - CES0500000002\n Primary Metal Manufacturing Average Weekly Hours of All Employees, Seasonally Adjusted - CES3133100002\n

In [121]:
# from BLS website
import requests
import json
import prettytable
import pandas as pd
headers = {'Content-type': 'application/json'}
data = {"seriesid": series_id,"startyear":"2015", "endyear":"2025"}
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=json.dumps(data), headers=headers)
json_data = json.loads(p.text)


In [122]:

df_lst=[]
titles=[]
for (title, series) in zip([key for key in data_series.keys()], json_data['Results']['series']):
    df = pd.DataFrame(series['data'])
    df[title] = df['value'].astype(str).astype(float)
    # df.rename(columns = {'value':title}, inplace=True)
    df['time'] = pd.to_datetime(df['year']+'-'+df['period'].str[-2:])
    df.set_index('time', inplace=True)
    df_lst.append(df[[ title]])
    titles.append(title)
# todo join df_dict by year and period.  
df_join = df_lst[0].join(df_lst[1:])
df_join.sort_index(ascending=True, inplace=True)
df_diff = df_join.diff()
df_join.reset_index(inplace=True)
df_diff.reset_index(inplace=True)

In [123]:
sec = [t for t in titles if 'TOTAL' not in t.upper()]
df_sec_diff=df_diff[['time']+sec]
df_melt = df_sec_diff.melt(id_vars=['time'], value_vars=sec)


In [134]:
df_sec_diff.loc[df_sec_diff.index[-1], sec].values

array([23.0, 36.0, 11.0, 1.7000000000000028, 74.0, 11.0, 5.0, -6.0, 8.0,
       112.0, 15.0, 45.19999999999891], dtype=object)

In [140]:
fig = px.area(df_melt, x='time', y='value', color='variable', title="None Farm Payroll", width=1200, height=600)
fig.show()

In [42]:
df_lst[1]

,time,Manufacturing
0,2023-12-01,12956
1,2023-11-01,12948
2,2023-10-01,12923
3,2023-09-01,12954
4,2023-08-01,12941
...,...,...
115,2014-05-01,12146
116,2014-04-01,12134
117,2014-03-01,12120
118,2014-02-01,12106


,Total Nonfarm,Manufacturing,Total Private,Government,Construction,Oil and Gas Extraction,Professional and Business Services,Leisure and Hospitality,Other Services,Mining and Logging,Financial Activities,Private Education and Health Services,Information
time,,,,,,,,,,,,,
2023-12-01,157347,12956,134292,23055,8126,118.2,22904,16813,5863,641,9240,25831,3017
2023-11-01,157014,12948,134014,23000,8102,117.4,22869,16775,5864,640,9227,25747,2999
2023-10-01,156832,12923,133862,22970,8087,117.8,22859,16765,5854,644,9223,25637,2982
2023-09-01,156667,12954,133764,22903,8065,117.6,22864,16708,5855,645,9223,25560,3008
2023-08-01,156421,12941,133568,22853,8052,117.4,22865,16681,5846,644,9223,25479,2997
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-05-01,138500,12146,116668,21832,6109,196.7,19093,14670,5568,886,7951,21364,2717
2014-04-01,138298,12134,116445,21853,6084,195.2,19053,14614,5565,885,7944,21321,2721
2014-03-01,137987,12120,116154,21833,6038,193.3,18994,14564,5553,879,7932,21294,2725
